In [1]:
import numpy as np
import scipy.sparse
from scipy.sparse.linalg import lobpcg, eigsh, minres, LinearOperator
from scipy.sparse import csr_matrix
import time
from utils.tools import build_weighted_bethe_hessian

In [2]:
adj_path = "/Users/i.lobov/hyperwords/data/wiki/wikipedia.corpus.nodups_counts_win=1.adj"
adjacency_matrix = scipy.sparse.load_npz(adj_path + ".npz")
#adjacency_matrix.data = 1 + np.log(adjacency_matrix.data)
adjacency_matrix.data = adjacency_matrix.data.astype(np.float32) ** 0.3
#adjacency_matrix.data /= np.max(adjacency_matrix.data)
degrees = np.asarray(adjacency_matrix.sum(axis=1)).flatten()

In [6]:
#unique_vals = np.unique(adjacency_matrix.data)

In [3]:
#r = np.sqrt(5176.292450)
r = np.sqrt(np.mean(degrees**2) / np.mean(degrees) - 1)
#r = np.sqrt(np.mean(degrees))
Hr = build_weighted_bethe_hessian(adjacency_matrix, r)

#I = scipy.sparse.eye(n, format='csr')
#Hr = D - adjacency_matrix #+ I*np.mean(degrees)

In [8]:
# n = adjacency_matrix.shape[0]
# I = scipy.sparse.eye(n, n, dtype=np.float32, format='csr')
# Hr = Hr + I * 55

In [5]:
# preconditioner = scipy.sparse.spdiags(1.0 / bethe_diagonal, [0], n, n, format='csr')

In [6]:
# n = adjacency_matrix.shape[0]
# dim = 100
# tol = np.sqrt(1e-15)*n

# start = time.time()
# vals, vecs = eigsh(Hr, dim, which='SA', tol=tol)
# print("time elapsed: %d" % (time.time() - start))

In [4]:
class Operator(object):

    def __init__(self, A):
        self.A = A.astype(PETSc.ScalarType)
        self.n_calls = 0

    def mult(self, A, x, y):
        xx = x.getArray(readonly=1)
        yy = y.getArray(readonly=0)
        yy[:] = self.A.dot(xx)
        self.n_calls += 1

In [5]:
from petsc4py import PETSc
from slepc4py import SLEPc

n = adjacency_matrix.shape[0]
mat = Operator(Hr)
A = PETSc.Mat().createPython([n, n], mat)
A.setUp()

k = 500
#tol = np.sqrt(1e-15)*n
tol = 1e-4
max_iter = 1

E = SLEPc.EPS()
E.create()
#E.setType(SLEPc.EPS.Type.LANCZOS)
E.setOperators(A)
E.setProblemType(SLEPc.EPS.ProblemType.HEP)
E.setDimensions(k)
E.setTolerances(tol, max_iter)
E.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_REAL)
#E.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_MAGNITUDE)

In [6]:
import time

mat.n_calls = 0
start = time.time()
E.solve()
print("Time elapsed: %f" % (time.time() - start))
print("Number of calls to Ax: %d" % mat.n_calls)

Time elapsed: 121.953254
Number of calls to Ax: 500


In [9]:
diffs = np.sort(vals-55) - np.sort(np.load("../data/wiki/win=1_weighted_bethe_hessian_slepc_pow=0.3_dim=500.vals.npy"))

ValueError: operands could not be broadcast together with shapes (81,) (505,) 

In [24]:
np.histogram(diffs)

(array([62, 36,  1,  0,  0,  0,  0,  0,  0,  1]),
 array([ -3.23712826e-04,   1.24752522e-05,   3.48663330e-04,
          6.84851408e-04,   1.02103949e-03,   1.35722756e-03,
          1.69341564e-03,   2.02960372e-03,   2.36579180e-03,
          2.70197988e-03,   3.03816795e-03]))

In [10]:
print("")
its = E.getIterationNumber()
print("Number of iterations of the method: %i" % its)
sol_type = E.getType()
print("Solution method: %s" % sol_type)
nev, ncv, mpd = E.getDimensions()
print("Number of requested eigenvalues: %i" % nev)
tol, maxit = E.getTolerances()
print("Stopping condition: tol=%.4g, maxit=%d" % (tol, maxit))
#nconv = E.getConverged()
nconv = 500
print("Number of converged eigenpairs: %d" % nconv)

vecs = np.zeros([n, nconv])
vals = np.zeros(nconv)

xr, tmp = A.getVecs()
xi, tmp = A.getVecs()

if nconv > 0:
    print("")
    print("        k          ||Ax-kx||/||kx|| ")
    print("----------------- ------------------")
    for i in range(nconv):
        k = E.getEigenpair(i, xr, xi)
        vals[i] = k.real
        vecs[:,i] = xr
        if i < 10:
            error = E.computeError(i)
            if k.imag != 0.0:
                print(" %9f%+9f j  %12g" % (k.real, k.imag, error))
            else:
                print(" %12f       %12g" % (k.real, error))
    print("")


Number of iterations of the method: 1
Solution method: krylovschur
Number of requested eigenvalues: 500
Stopping condition: tol=0.0001, maxit=1
Number of converged eigenpairs: 500

        k          ||Ax-kx||/||kx|| 
----------------- ------------------
   -52.814198          0.0267771
   -11.473175        2.86993e-11
    -7.327230           0.193008
    -5.684050                  0
    -5.014655        1.21125e+09
    -4.622478                 -0
    -4.379625                  0
    -3.950231                nan
    -3.509568                nan
    -3.314204                nan


Error: error code 63
[0] EPSGetEigenpair() line 397 in /Users/i.lobov/slepc-3.8.2/src/eps/interface/epssolve.c
[0] Argument out of range
[0] Argument 2 out of range

In [25]:
output_path = "../data/wiki/win=1_weighted_bethe_hessian_slepc_PSD_pow=0.3_dim=100"
np.save(output_path + ".vecs", vecs)
np.save(output_path + ".vals", vals - 55)
np.save(output_path + ".degrees", degrees)

In [14]:
output_path = "../data/wiki/win=1_weighted_bethe_hessian_scaled3_pow=0.30_dim=100"
np.save(output_path + ".vecs", vecs2)
np.save(output_path + ".vals", vals2 - np.mean(degrees))
np.save(output_path + ".degrees", degrees)

In [11]:
output_path = "../data/wiki/win=1_bethe_hessian_small_rhoB_est_pow=0.00_dim=100"
vecs2 = np.load(output_path + ".vecs.npy")
vals2 = np.load(output_path + ".vals.npy")

In [14]:
all_vals = np.concatenate([vals, vals2], axis=0)
all_vecs = np.concatenate([vecs, vecs2], axis=1)

In [19]:
top_vals_inds = np.argsort(all_vals)[:100]
output_path = "../data/wiki/win=1_bethe_hessian_combo_rhoB_est_pow=0.00_dim=100"
np.save(output_path + ".vecs", all_vecs[:, top_vals_inds])
np.save(output_path + ".vals", all_vals[top_vals_inds])
np.save(output_path + ".degrees", degrees)